In [3]:
# !gdown https://drive.google.com/file/d/1lhAaeQCmk2y440PmagA0KmIVBIysVMwu/view?usp=drive_link

zsh:1: no matches found: https://drive.google.com/file/d/1lhAaeQCmk2y440PmagA0KmIVBIysVMwu/view?usp=drive_link


Start code

In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import models,transforms
import json
import cv2
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Create torch dataset

In [ ]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        # super().__init__()
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485 ,0.456, 0.406],std=[0.229, 0.224, 0.225])
            ]
        )

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]
        #cv2 reads image in bgr format but we need it in rgb
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        #kps are of orignal size (not 224x224) but we need to resize kps such that they map to the transformed 224x224 img
        #simple cross mul w :240 :: 224 : ?
        kps[::2] *= 224.0/w #adjust x coords of kps
        kps[1::2] *= 224.0/h #adjust y coords of kps
        
        return img,kps

In [ ]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
validation_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset,batch_size=8,shuffle=True)
validation_loader = DataLoader(validation_dataset,batch_size=8,shuffle=True)

CREATE MODEL

In [ ]:
model =models.resnet50(pretrained=True)
#resnet 50 is not trained on keypoint extraction, hence fine tune it 
#fc is fully connected layer
model.fc = torch.nn.Linear(model.fc.in_features,14*2) #replaces last layer of nn with 28 reg heads
model = model.to(device)

TRAINING THE MODEL

In [ ]:
#defining loss function
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(lr = 0.001))
num_epochs = 25
for epoch in range(num_epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs,kps)
        loss.backward()
        optimizer.step()

        if i % 5 == 0:
            print(f"Epoch: {epoch}, iter : {i}, loss: {loss.item()}")

In [ ]:
torch.save(model.state_dict(),"keypoints_model.pth")